# Random Forest Regression for Predicting VHH Aggregation

Author: Chloe Brown (chloebr@umich.edu)

Objective: The goal of this notebook is to use Random Forest Regressiont to predict the aggregation properties (% monomer and % aggregate) of VHHs. 

Packages and Versions Used:
1. Python (version 3.9.18)
2. Scikit-learn (version 1.5.2)
3. Numpy (version 1.26.3)
4. Pandas (version 2.2.3)

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate, LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

## **Part I: Input**
Fields to Edit: 
1. **feature_input_path** = file with physicochemical properties from MOE for training dataset. 
2. **property_input_path** = file with experimental data for training dataset.
3. **test_data_file** = file with physicochemical properties from MOE for nanobodies you would like to test. There should also be a column called "VHH Code" with the id's associated with each sample.
4. **output_file** = file to save results.


In [2]:
# fields to edit

feature_input_file = "molecular_features.csv"
property_input_file = "sec_data.csv"
test_data_file = "input.csv"
output_file = "results.csv"

In [3]:
# features list
comp_cols = ['pI_seq', 'zquadrupole', 'zdipole', 'zeta', 'hyd_moment', 'dipole_moment', 'net_charge', 'asa_hph', 
             'asa_hyd', 'asa_vdw', 'pI_3D', 'patch_cdr_ion_n', 'patch_cdr_ion_1', 'patch_cdr_ion', 'patch_cdr_neg_n', 
             'patch_cdr_neg_1', 'patch_cdr_neg', 'patch_cdr_pos_n', 'patch_cdr_pos_1', 'patch_cdr_pos', 'patch_cdr_hyd_n',
             'patch_cdr_hyd_1', 'patch_cdr_hyd', 'patch_ion_%', 'patch_ion_n', 'patch_ion_1', 'patch_ion',
             'patch_neg_%', 'patch_neg_n', 'patch_neg_1', 'patch_neg', 'patch_pos_%', 'patch_pos_n', 'patch_pos_1', 
             'patch_pos', 'patch_hyd_%', 'patch_hyd_n', 'patch_hyd_1', 'patch_hyd']

# import data
properties = pd.read_csv(property_input_file, usecols = ['HMW_up', 'Main_up'])
comp_feat = pd.read_csv(feature_input_file).drop(labels = "VHH Code", axis = 1)
comp_feat = comp_feat[comp_cols]

# scale data
scaler = MinMaxScaler()
comp_feat = pd.DataFrame(scaler.fit_transform(comp_feat), columns = comp_feat.columns)

# read in test data
vhh = pd.read_csv(test_data_file)

# make a df for the results
results_df = pd.DataFrame(vhh['VHH Code'])

# data preperation
vhh = vhh[comp_cols]
vhh_scaled = pd.DataFrame(scaler.transform(vhh), columns = vhh.columns)

## **Part 2: % monomer regressor**


In [ ]:
# train the model
X = comp_feat
y = properties['Main_up']

# cross validation
rf = RandomForestRegressor(max_features = .7, n_estimators = 5, random_state = 10,
                           max_depth = 5, min_samples_split = 3)
rf_cv = cross_validate(rf, X=X, y=y, cv = LeaveOneOut(), 
                       scoring = 'neg_root_mean_squared_error')
print(f"RMSE from cross validation: {-rf_cv['test_score'].mean():1.3f}")

# fit and evaluate
rf.fit(X, y)
rf_pred_e = rf.predict(X)
print(f"RMSE: {mean_squared_error(y, rf_pred_e, squared = False):1.3f}")
print(f"MAE: {mean_absolute_error(y, rf_pred_e):1.3f}")

In [ ]:
# predict % monomer
vhh_predictions = rf.predict(vhh_scaled)
results_df['Main_up'] = vhh_predictions
results_df

## **Part 3: % aggregate regressor**


In [ ]:
# train the model
X = comp_feat
y = properties['HMW_up']

# cross validation
rf = RandomForestRegressor(max_features = .1, n_estimators = 5, random_state = 1,
                           max_depth = 5, min_samples_split = 2)
rf_cv = cross_validate(rf, X=X, y=y, cv = LeaveOneOut(), 
                       scoring = 'neg_root_mean_squared_error')
print(f"RMSE from cross validation: {-rf_cv['test_score'].mean():1.3f}")

# fit and evaluate
rf.fit(X, y)
rf_pred_e = rf.predict(X)
print(f"RMSE: {mean_squared_error(y, rf_pred_e, squared = False):1.3f}")
print(f"MAE: {mean_absolute_error(y, rf_pred_e):1.3f}")

In [ ]:
# predict vhh
vhh_predictions = rf.predict(vhh_scaled)
results_df['HMW_up'] = vhh_predictions
results_df

## **Save Results**

In [ ]:
results_df.to_csv(output_file)